# Loading test data

In [207]:
import pyabf
import numpy as np
from utils import extract_channels

example = "/home/jouv/myodish_dropzone/ELN24917-045/abf/2024_06_05/platform2_bottom.abf"

# load the abf file
abf = pyabf.ABF(example)

data = extract_channels(abf)[3]

for _, row in data.iterrows():
    if row["signal_value"] != 0.0:
        data = data[_::]
        break

# create subset of 10000 points
data = data[:10000]

In [208]:
data.describe()

,time_index,signal_value
count,10000.000000,10000.000000
mean,968.945000,-11687.699219
std,28.868957,64.157791
min,918.950000,-12677.000000
25%,943.947500,-11729.000000
50%,968.945000,-11683.000000
75%,993.942500,-11642.000000
max,1018.940000,-11218.000000


Plot initial data

In [209]:
# plot the subset using plotly graph objects
import plotly.graph_objects as go
import numpy as np

def plot(data, x=None, y=None):
    if x is None:
        x = data["time_index"]
    if y is None:
        y = data["signal_value"]

    # define plot layout parameters
    layout = go.Layout(title=f'Channel',
        xaxis=dict(
            visible=False, 
            autorange=True,
            fixedrange=False, 
            showgrid=False, 
            #rangeslider=dict(visible=True), 
            type="linear"
        ),
        yaxis=dict(
            visible=True, 
            autorange=True,
            fixedrange=True,
            showgrid=True
        ),
        dragmode='pan',
        annotations=[],
        hovermode='x unified'
    )

    fig = go.Figure(layout=layout)

    fig.add_trace(go.Scatter(x=x, y=y, mode='lines'))

    fig.show()




In [210]:
layout = go.Layout(title=f'Channel',
        xaxis=dict(
            visible=False, 
            autorange=True,
            fixedrange=False, 
            showgrid=False, 
            #rangeslider=dict(visible=True), 
            type="linear"
        ),
        yaxis=dict(
            visible=True, 
            autorange=True,
            fixedrange=True,
            showgrid=True
        ),
        dragmode='pan',
        annotations=[],
        hovermode='x unified'
    )

# Test PeakDetection Class

In [224]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks, convolve, savgol_filter

class PeakDetection:
    """
    Class to process an .abf signal, computing baseline and floor signals, and detecting peaks in a signal
    """
    def __init__(self, data, time_column='time_index', signal_column='signal_value'):
        self.data = data
        self.time_column = time_column
        self.signal_column = signal_column
        self.smooth = self.smooth_signal()

    
    def compute_baseline(self, target="smooth", window_size=175, peak_prominence=0.2, smoothing_window_length=201, polyorder=3):
        """
        Function to compute the baseline signal
        :param window_size: int
        :param peak_prominence: float
        :param smoothing_window_length: int
        :param polyorder: int
        :return: list
        """
        if target == "smooth":
            y = np.array(self.smooth)
        elif target == "raw":
            y = np.array(self.data[self.signal_column])
        else:
            raise ValueError("target must be either 'smooth' or 'raw'")
        
        end_point = len(y) - window_size
        floor = np.empty([end_point])

        for i in range(end_point):
            frame = y[i:i + window_size]

            valleys, _ = find_peaks(-frame)

            if len(valleys) > 0:
                floor[i] = frame[valleys].min()
            else:
                floor[i] = np.nan  
                
        padding = smoothing_window_length // 2

        # Pad the resulting signal
        floor_padded = np.concatenate([floor[:padding], floor, floor[-padding:]]) # remove padding

        # Smooth the signal
        floor_smooth = savgol_filter(floor_padded, window_length=smoothing_window_length, polyorder=polyorder, mode='nearest')

        return floor_smooth.tolist()
    
    
    def compute_ceiling(self, target="smooth", window_size=175, peak_prominence=0.2, smoothing_window_length=201, polyorder=3):
        """
        Function to compute the ceiling signal
        :param window_size: int
        :param peak_prominence: float
        :param smoothing_window_length: int
        :param polyorder: i nt
        :return: list
        """
        if target == "smooth":
            y = np.array(self.smooth)
        elif target == "raw":
            y = np.array(self.data[self.signal_column])
        else:
            raise ValueError("target must be either 'smooth' or 'raw'")

        end_point = len(y) - window_size
        ceiling = np.empty([end_point])

        for i in range(end_point):
            frame = y[i:i+ window_size]
            
            # Find the peaks in the frame
            peaks, _ = find_peaks(frame)
            
            # If there are any peaks, use the highest peak as the ceiling for this window
            if len(peaks) > 0:
                ceiling[i] = frame[peaks].max()
            else:
                ceiling[i] = np.nan  # No ceiling defined if there are no peaks

        padding = smoothing_window_length // 2
        
        # Pad the resulting signal
        ceiling_padded = np.concatenate([ceiling[:padding], ceiling, ceiling[-padding:]])

        # Smooth the signal
        ceiling_smooth = savgol_filter(ceiling_padded, window_length=smoothing_window_length, polyorder=polyorder, mode='nearest')

        return ceiling_smooth.tolist()
    

    def smooth_signal(self, smoothing_window_length=51, polyorder=7):
        """
        Function to smooth the raw signal
        :param window_length: int
        :param polyorder: int
        :return: list
        """
        y = np.array(self.data[self.signal_column])

        # Pad the signal
        padding = smoothing_window_length // 2
        y_padded = np.concatenate([y[:padding], y, y[-padding:]])

        # Smooth the signal using a Savitzky-Golay filter
        smoothed_signal = savgol_filter(y_padded, window_length=smoothing_window_length, polyorder=polyorder, mode='nearest')

        # Remove the padding
        smoothed_signal = smoothed_signal[padding:-padding]

        return smoothed_signal.tolist()

In [225]:
# create a peak detection object
obj = PeakDetection(data)

Baseline

In [222]:
baseline = obj.compute_baseline()
plot(data, y=baseline)

TypeError: list indices must be integers or slices, not str

In [204]:
ceiling = obj.compute_ceiling()
plot(data, y=ceiling)

In [205]:
smoothed_signal = obj.smooth_signal()

In [223]:
# plot the signal with the baseline and ceiling in the same figure
fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter(x=data["time_index"], y=data["signal_value"], mode='lines', name='Signal'))
fig.add_trace(go.Scatter(x=data["time_index"], y=baseline, mode='lines', name='Baseline'))
fig.add_trace(go.Scatter(x=data["time_index"], y=ceiling, mode='lines', name='Ceiling'))
#fig.add_trace(go.Scatter(x=data["time_index"], y=smoothed_signal, mode='lines', name='Smoothed Signal'))
fig.show()